In [ ]:
import datetime
import pandas as pd
import gymnasium as gym
import gym_anytrading
import numpy as np
import datetime
from gym_anytrading.envs import StocksEnv
import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout 
from keras.optimizers import Adam
import subprocess

In [2]:
df = gym_anytrading.datasets.STOCKS_GOOGL.copy()
df.shape

(2335, 6)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2335 entries, 2009-05-22 to 2018-08-29
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2335 non-null   float64
 1   High       2335 non-null   float64
 2   Low        2335 non-null   float64
 3   Close      2335 non-null   float64
 4   Adj Close  2335 non-null   float64
 5   Volume     2335 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 127.7 KB


In [4]:
from finta import TA
df['TEMA'] = TA.TEMA(df, 30) # Triple Exponential Moving Average --> in ce directie se indreapta pretul pe parcursul a 12 zile cu accent mare pe ultimele zile
df['ER'] = TA.ER(df) # Kaufman Efficiency Indicator --> confirmare a trendului (-1,1) : -1 downtrend, 1 uptrend, aprox 0 random (trend confirmation)
df['RSI'] = TA.RSI(df) # Relative Strength Index --> intre 0 si 100, 30-70 intervalul de interes, 30-0 oversold, 70-100 overbought (trend confirmation)
df['OBV'] = TA.OBV(df) # On Balance Volume --> confirmare a trendului (trend confirmation) --> se bazeaza pe volumul tranzactiilor
df['STOCH'] = TA.STOCH(df) # Stochastic Oscillator --> confirmare a trendului (trend confirmation) --> intre 0 si 100, 20-80 intervalul de interes, 20-0 oversold, 80-100 overbought
df.fillna(0, inplace=True)
df.head(20)

,Open,High,Low,Close,Adj Close,Volume,TEMA,ER,RSI,OBV,STOCH
Date,,,,,,,,,,,
2009-05-22,198.528534,199.524521,196.196198,196.946945,196.946945,3433700,196.946945,0.000000,0.000000,0.0,0.000000
2009-05-26,196.171173,202.702698,195.195190,202.382385,202.382385,6202700,201.768658,0.000000,100.000000,6202700.0,0.000000
2009-05-27,203.023026,206.136139,202.607605,202.982986,202.982986,6062500,203.105999,0.000000,100.000000,12265200.0,0.000000
2009-05-28,204.544540,206.016022,202.507507,205.405411,205.405411,5332200,204.966986,0.000000,100.000000,17597400.0,0.000000
2009-05-29,206.261261,208.823822,205.555557,208.823822,208.823822,5291100,207.561945,0.000000,100.000000,22888500.0,0.000000
2009-06-01,209.574570,215.015015,209.474472,213.493500,213.493500,6638100,211.077229,0.000000,100.000000,29526600.0,0.000000
2009-06-02,213.338333,215.195190,211.911911,214.414413,214.414413,5241900,213.421299,0.000000,100.000000,34768500.0,0.000000
2009-06-03,213.213211,216.446442,212.212219,216.041046,216.041046,7058500,215.420351,0.000000,100.000000,41827000.0,0.000000
2009-06-04,217.867874,220.840836,217.467468,220.360367,220.360367,7268900,218.225981,0.000000,100.000000,49095900.0,0.000000


In [5]:
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv
from environment import CustomTradingEnv
from gym_anytrading.envs import StocksEnv
import torch
torch.set_num_threads(1)
env = CustomTradingEnv(df=df,window_size=30,frame_bound=(30,int(len(df)*.8)))
print(env._get_info())
vectorized_env = DummyVecEnv([lambda: env])
model = RecurrentPPO('MlpLstmPolicy', vectorized_env,  verbose=0,seed=42)
model.learn(total_timesteps=int(2e4),progress_bar=True)

{'total_reward': None, 'total_profit': None, 'position': None}


Output()

In [6]:
# evaluate the agent
from environment import CustomTradingEnv
from tqdm import tqdm
def evaluate_agent(df,model,episodes):
    test_env = CustomTradingEnv(df=df, frame_bound=(int(0.8*len(df)), len(df)), window_size=30)
    mean_reward, mean_profit = 0, 0
    for episode in tqdm(range(episodes)):
        state = test_env.reset()[0]
        while True:
            action, x = model.predict(state)
            state, reward, deprecated,done, info = test_env.step(action)
            if done:
                print(info)
                break
        mean_profit += info['total_profit']
        mean_reward += info['total_reward']
        
    mean_reward /= episodes
    mean_profit /= episodes
    
    return mean_reward, mean_profit


In [8]:
# test reward function
from tqdm import tqdm
def evaluate_agent(df,model,episodes):
    test_env = CustomTradingEnv(df=df, frame_bound=(int(0.8*len(df)), len(df)), window_size=30)
    mean_reward, mean_profit = 0, 0
    profits = []
    rewards = []
    for episode in tqdm(range(episodes)):
        state = test_env.reset()[0]
        while True:
            action, x = model.predict(state)
            state, reward, deprecated,done, info = test_env.step(action)
            if done:
                print(info)
                break
        mean_profit += info['total_profit']
        mean_reward += info['total_reward']
        profits.append(info['total_profit'])
        rewards.append(info['total_reward'])
        
    mean_reward /= episodes
    mean_profit /= episodes
    
    return mean_reward, mean_profit,mean_profit/mean_reward,profits,rewards

mean_reward, mean_profit,fr,profits,rewards=evaluate_agent(df,model,episodes=100)

profits = np.array(profits)
rewards = np.array(rewards)
import seaborn as sns
sns.scatterplot(x=profits,y=rewards)

  1%|          | 1/100 [00:00<01:36,  1.02it/s]

{'total_reward': 459.90008699999987, 'total_profit': 1.2473377137196848, 'position': <Positions.Long: 1>}


  2%|▏         | 2/100 [00:01<01:30,  1.08it/s]

{'total_reward': 420.8500340000003, 'total_profit': 1.2471017502256343, 'position': <Positions.Long: 1>}


  3%|▎         | 3/100 [00:02<01:25,  1.13it/s]

...

 99%|█████████▉| 99/100 [01:33<00:00,  1.18it/s]

{'total_reward': 417.9699700000001, 'total_profit': 1.3450653495482114, 'position': <Positions.Long: 1>}


100%|██████████| 100/100 [01:34<00:00,  1.06it/s]

{'total_reward': 445.28015000000005, 'total_profit': 1.2255203774184324, 'position': <Positions.Long: 1>}


(419.77743372, 1.228942131120744)

In [24]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from environment import CustomTradingEnv
from gym_anytrading.envs import StocksEnv
import torch
torch.set_num_threads(1)
env = CustomTradingEnv(df=df,window_size=30,frame_bound=(30,int(len(df)*.8)))
print(env._get_info())
vectorized_env = DummyVecEnv([lambda: env])
model_ppo = PPO('MlpPolicy', vectorized_env,  verbose=0)
model_ppo.learn(total_timesteps=int(6e4),progress_bar=True)

Output()

{'total_reward': None, 'total_profit': None, 'position': None}


In [25]:
evaluate_agent(df,model_ppo,episodes=10)

 20%|██        | 2/10 [00:00<00:01,  5.67it/s]

{'total_reward': 323.7200280000003, 'total_profit': 0.8905060747762259, 'position': <Positions.Long: 1>}
{'total_reward': 361.11999200000037, 'total_profit': 0.9360545557819862, 'position': <Positions.Long: 1>}


 40%|████      | 4/10 [00:00<00:00,  6.19it/s]

{'total_reward': 589.7497560000002, 'total_profit': 1.0672091195281281, 'position': <Positions.Long: 1>}
{'total_reward': 408.9300539999997, 'total_profit': 0.9183090106119278, 'position': <Positions.Long: 1>}


 60%|██████    | 6/10 [00:00<00:00,  6.40it/s]

...

 80%|████████  | 8/10 [00:01<00:00,  6.35it/s]

{'total_reward': 379.6599680000004, 'total_profit': 1.0963290282766847, 'position': <Positions.Long: 1>}
{'total_reward': 296.88000200000033, 'total_profit': 0.9869246309034023, 'position': <Positions.Long: 1>}


100%|██████████| 10/10 [00:01<00:00,  6.20it/s]

{'total_reward': 370.3800679999997, 'total_profit': 0.9769565140887927, 'position': <Positions.Long: 1>}
{'total_reward': 524.7901609999999, 'total_profit': 1.0067141789394867, 'position': <Positions.Long: 1>}


(422.1689626000001, 0.9636973287206312)